In [1]:
import pandas as pd
import numpy as np
from src.data_collection import deg_utils, stringdb_api
from src.analysis import clustering_utils
import networkx as nx
from src.graph.graph_util import *
import config.config as config
from src.visualization.plots import plot_dist
from cdlib import algorithms
from IPython.display import Image, display

import dill
dill.load_session("02_session.pkl")

Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'bayanpy', 'graph_tool', 'leidenalg', 'wurlitzer'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'infomap', 'wurlitzer'}


# Clustering Analysis

I will employ the following Community Discovery algorithms:

- **Angel (Percolation)**, node-centric bottom up approach. Use the assumption that locally each node is able to identify its community but globally we are tangled in complex overlaps (this is generally true for PPI networks). Quoting the docs "Angel is the, faster, successor of Demon."
- **Infomap (Entity Closeness)**, similar to Louvain, exploits the notions of conductance, minimum descriptive lengths and random walkers. Low conductance means few bridges thus well separated communities
- **Girvan-Newman (Bridge Detection)**, remove bridges to create partitions. The output is a dendrogram which can be cut based on a user specified threshold.

## Loading the networks

In [5]:
downregulated_graph = get_gene_graph([], config.LIVER_DOWNREGULATED_GRAPH_FILE_NAME)
upregulated_graph = get_gene_graph([], config.LIVER_UPREGULATED_GRAPH_FILE_NAME)
combined_graph = nx.compose(downregulated_graph, upregulated_graph)
combined_graph.number_of_nodes(), combined_graph.number_of_edges()

Loaded graph '/home/emiliano/Desktop/Università/Network Analysis/Project/Code/data/graphs/liver_downregulated_graph': 6858 nodes, 21862 edges
Loaded graph '/home/emiliano/Desktop/Università/Network Analysis/Project/Code/data/graphs/liver_upregulated_graph': 5871 nodes, 17408 edges


(8795, 32313)

In [12]:
result = stringdb_api.get_gene_set_enrichment(demon.communities[1], background_genes)
res_filtered = [r for r in result if r["category"] in ["KEGG", "Function"] and r["number_of_genes_in_background"] > 2]
res_sorted = sorted(res_filtered, key=lambda x: (x['fdr'], -(x["number_of_genes"]/x["number_of_genes_in_background"])), reverse=False)
res_sorted

[{'category': 'KEGG',
  'term': 'mmu05204',
  'number_of_genes': 62,
  'number_of_genes_in_background': 99,
  'ncbiTaxonId': 10090,
  'inputGenes': ['Gstt1',
   'Gstt3',
   'Cyp2c29',
   'Gstm5',
   'Gstm3',
   'Gstm7',
   'Gstm1',
   'Cyp2b13',
   'Gstm2',
   'Hsd11b1',
   'Cyp1b1',
   'Cyp2c39',
   'Gsto1',
   'Cyp2c23',
   'Cyp2e1',
   'Gsta3',
   'Mgst3',
   'Gstm4',
   'Ugt2b34',
   'Ugt2b1',
   'Ugt2b35',
   'Ugt2a3',
   'Cyp3a16',
   'Cyp3a13',
   'Gstk1',
   'Hpgds',
   'Cyp1a2',
   'Gsta2',
   'Gsta4',
   'Ptgs2',
   'Cyp3a11',
   'Gstp2',
   'Ephx1',
   'Cyp2c54',
   'Cbr1',
   'Gsto2',
   'Cyp2c70',
   'Cyp3a25',
   'Ugt2b5',
   'Cyp3a44',
   'Cyp2b10',
   'Ugt2b38',
   'Ugt1a1',
   'Ugt2b37',
   'Cyp2c50',
   'Cyp2b9',
   'Ugt2b36',
   'Gsta1',
   'Mgst2',
   'Cyp2c68',
   'Sult1a1',
   'Gstm6',
   'Sult2a1',
   'Ugt1a6a',
   'Ugt1a6b',
   'Ugt1a10',
   'Mgst1',
   'Gstt4',
   'Gstp1',
   'Gstp3',
   'Cyp1a1',
   'Gstt2'],
  'preferredNames': ['Gstt1',
   'Gstt3',
   'Cyp2c

In [13]:
best_kegg = next((r for r in res_sorted if r["category"] == "KEGG"), None)
best_function = next((r for r in res_sorted if r["category"] == "Function"), None)

In [16]:
best_kegg["category"], best_kegg["term"], best_kegg["description"], best_kegg["fdr"], best_kegg["number_of_genes"], best_kegg["number_of_genes_in_background"]

('KEGG', 'mmu05204', 'Chemical carcinogenesis', 3.02e-56, 62, 99)

In [17]:
best_function["category"], best_function["term"], best_kegg["description"], best_function["fdr"], best_function["number_of_genes"], best_function["number_of_genes_in_background"]

('Function', 'GO:0016491', 'Chemical carcinogenesis', 3.19e-49, 116, 787)

In [11]:
background_genes = {data["name"] for node, data in combined_graph.nodes(data=True)}
background_genes

{'Srsf3',
 'Actb',
 'Fundc1',
 'Ripk3',
 'Ppp1r3c',
 'Asz1',
 'Hsdl2',
 'Casc1',
 'Ggt5',
 'Fblim1',
 'Id1',
 'Chst2',
 'Med31',
 'Osbpl5',
 'Dao',
 'Sec11c',
 'Cfap298',
 'Eapp',
 'Igfbp7',
 'Gnal',
 'Npas4',
 'Alpl',
 'Chchd3',
 'Ddx42',
 'Zfp521',
 'Cct6a',
 'Nexmif',
 'Pih1d3',
 'Trappc3',
 'Kcne4',
 'Serpinb9c',
 'Flt4',
 'Adora1',
 'Kpnb1',
 'Flnb',
 'Gpr19',
 'Bglap3',
 'Snx4',
 'Ap4s1',
 'Icos',
 'Prdm16',
 'Lpar3',
 'Cdt1',
 'Tdp2',
 'Atp6v1b2',
 'P2rx1',
 'Gbp7',
 'Chmp4c',
 'Pacsin2',
 'Ap4b1',
 'Il22ra2',
 'Stard5',
 'Plpp5',
 'Psmc4',
 'Cd200r4',
 'Sgsm1',
 'Smpd4',
 'Slc39a14',
 'Tmem67',
 'Gstt2',
 'Pus1',
 'Esm1',
 'Mdk',
 'Bhmt',
 'Mettl27',
 'Cyp3a41a',
 'Glp2r',
 'Slc9a1',
 'Klrc3',
 'Cd244a',
 'Mical1',
 'Mpp2',
 'Rxrb',
 'Nr1h3',
 'Dele1',
 'Slc8a3',
 'Msmo1',
 'Cd8a',
 '4932438A13Rik',
 'Kif18a',
 'Nek4',
 'Plek',
 'Lyrm4',
 'Ndufaf4',
 'Bub1',
 'Pip5k1b',
 'Plxnb3',
 'Il11ra2',
 'Rgs2',
 'Bud31',
 '9530077C05Rik',
 'Cftr',
 'Npr2',
 'Cdkn2c',
 'Lats1',
 'Rtcb',
 

# Evaluation Idea
For each partition provided by the algorithms, i am going to perform **gene set enrichment** (for Pathways (KEGG) and Gene Ontology Function) and evaluate the enrichment by taking into account:
- **False Discovery Rate** (FDR) - the lowest the better - measures how unlikely it is thata the overlap occurred by chance. It is a statistical significance measure, kind like an adjuste p-value.
- **Number of background genes > 2** - to avoid small background that might mess the enrichment ratio
- **Enrichment Ratio** (n_genes / n_genes_in_background) - this measures how strong is the enrichment for that particular category. n_genes define the number of genes from the background (the partition) that belongs to that specific pathway/function.

I will select the enrichment that shows the lowest FDR and the highest Enrichment Ratio for each of the two categories i chose (KEGG and Function).

At the end im going to inspect the top k partitions to get a feel of how the algorithm performed and then compute the average FDR and Enrichment Ration of all the partitions in order to compare the 3 algorithms.

## Percolation (Angel)

In [23]:
angel = algorithms.angel(combined_graph, threshold=0.25)

In [7]:
angel.internal_edge_density().min

0.0019805035755945458

In [2]:
metadata = clustering_utils.get_partitioning_metadata(angel, background_genes, verbose=True, debug=True)

{'category': 'KEGG', 'term': 'mmu01100', 'number_of_genes': 1095, 'number_of_genes_in_background': 1534, 'ncbiTaxonId': 10090, 'inputGenes': ['Cox5a', 'Sdhd', 'Th', 'Gmpr', 'Alox12', 'Dbt', 'Hk2', 'Haao', 'Dbh', 'Aox1', 'Ftcd', 'Ocrl', 'Agpat3', 'Cyp51', 'Gstt1', 'Gstt3', 'Tat', 'Folh1', 'Blvra', 'Acp2', 'Ndufa11', 'Nt5c1b', 'Uck1', 'Gnmt', 'Tbxas1', 'Cyp2c29', 'Inmt', 'Prodh', 'Itpkc', 'Nqo1', 'Gys1', 'Gstm5', 'Gstm3', 'Gstm7', 'Gstm1', 'Hmox2', 'Dnmt1', 'Adh1', 'Eno2', 'Lbr', 'Nmrk2', 'G6pc2', 'Hmox1', 'Pafah1b3', 'Ndufs3', 'Cyp2b13', 'Cyp2a5', 'Mthfd2', 'Cs', 'Ampd3', 'Adh5', 'Smpd4', 'Inpp5k', 'Ggt1', 'Gcat', 'Elovl1', 'Tmem189', 'Srm', 'Mvd', 'Aprt', 'Qars', 'Pip4k2a', 'B4galnt1', 'Tph2', 'Acat2', 'Neu1', 'Akr1b7', 'Hmgcll1', 'Cers4', 'Fut1', 'St6galnac1', 'P4ha1', 'Pla2g12b', 'Sdhb', 'Kdsr', 'Tktl1', 'Acaa1b', 'Plcd1', 'Fads1', 'Apip', 'Pdhx', 'Pgam1', 'Gstm2', 'Adh4', 'Ndufs2', 'Cad', 'Pten', 'Ehmt2', 'Pip4k2c', 'Pigl', 'Ndufa2', 'Ldhc', 'Acsf3', 'Galns', 'Ptgds', 'Aldh1a3', 'Ac

In [3]:
clustering_utils.plot_partitioning_enrichment(metadata)

ValueError: num must be an integer with 1 <= num <= 2, not 0

<Figure size 640x480 with 0 Axes>

## Entity Closeness (Infomap)

In [6]:
infomap = algorithms.infomap(combined_graph)

In [7]:
infomap.internal_edge_density()

FitnessResult(min=0, max=1.0, score=np.float64(0.22678718665180245), std=np.float64(0.24714779107218032))

## Bridge Detection (Girvan-Newman)

In [10]:
girvan_newman = algorithms.girvan_newman(combined_graph, level=5)

KeyboardInterrupt: 

In [30]:
dill.dump_session("02_session.pkl")